# IBM Data Science Capstone Project

**Hello and Welcome!**

This Notebook will be used solely for IBM Data Science Capstone Project! I wish a great day and a lot of luck reviewing my results!
<br/>
Below you can find couple of my favourite jokes about being Data Scientist to lighten your mood:

***

>*The data science motto: If at first you don’t succeed; call it version 1.0*

***

>*Why should you take a data scientist with you into the jungle? <br/>- They can take care of Python problems*

***

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
